In [ ]:
# # code required to run on a fresh install or in google colab
# ! git clone https://github.com/CPMpy/XCP-explain.git /tmp/XCP-explain
# ! cd XCP-explain && git checkout ecai24
# ! pip install -r XCP-explain/requirements.txt
# ! pip install cpmpy

# # add XCP-explain to the Python path
# import sys
# root = "/tmp/XCP-explain"
# if root not in sys.path:
#     sys.path.insert(0, root)
root= "."

In [ ]:
%load_ext autoreload
%autoreload 2
"""
    Some imports used throughout the notebook
"""
import time
import os
from visualize import *

from cpmpy.transformations.normalize import toplevel_list
from factory import *
from read_data import get_data
from IPython.display import clear_output


import numpy as np
np.set_printoptions(linewidth=90)
# preload solvers
from cpmpy import SolverLookup
names = SolverLookup.solvernames()

## Explaning Optimization problems

In this notebook, we will consider the Nurse Rostering problem as an optimization problem. First we will focus on the following:
- "I **do not like** this solution" &#8592; Find **alternative** optimal or non-dominated solutions 
- "Why is there no **even better** solution?" &#8592; Find what **to change in my model** if I want a better solution?
- "What if **I want Y** in my solution **instead of X**?" &#8592; Find **how to change** the solutions or our optimization model



#### Load the optimization model

Same instance as in the previous notebook.


In [ ]:
instance = os.path.join(root,"Benchmarks/CustomInstance.txt")
data = get_data(instance)
factory = NurseSchedulingFactory(data)

Now as (multi-objective) optimization model! 3 different objectives in nurse rostering:

      - Penalty on positive shift requests

      - Penalty on negative shift requests

      - Penalty of the cover constraints

In [ ]:
model, nurse_view, penalty_on, penalty_off, penalty_cover = factory.get_multi_objective_model()

objectives = [penalty_on, penalty_off, penalty_cover] # penalty_on, penalty_off, penalty_cover are the 3 subobjectives

assert model.solve(solver="ortools") # you can try different solvers here!
opt_sol = nurse_view.value()
opt_value = model.objective_value()

display(visualize(opt_sol, factory))
print("Total penalty:", model.objective_value())
print("Time to calculate:", model.status().runtime, "s")

## Multiple solutions

- User not satisfied with optimal solution!!

- There could be multiple optimal solutions

- Find (a subset of) them by converting to a decision problem!

In [ ]:
opt_model = cp.Model(model.constraints) # init new model
opt_model += (model.objective_ == model.objective_value()) # force objective

opt_model.solveAll(solver="ortools", solution_limit=1,
                   display=lambda: display(visualize(nurse_view.value(), factory)))  # callback that visualizes sols

In [ ]:
# Convert to decision model enforcing the same objective value and use solveAll (with a limited number of solutions!)
new_model = cp.Model(model.constraints) # init new model
new_model += ... # force objective

sols_limit = ...

new_model.solveAll(solver="ortools", solution_limit=sols_limit,
                   display=lambda: display(visualize(nurse_view.value(), factory)))  # callback that visualizes sols

### Non-Dominated solutions

User not satisfied with optimal solution, considering bad the trade-offs it makes
- Optimize again, considering non-dominated solutions based on sub objectives!
- Non-dominated solution: No other solution will be better (lower penalty) on all objectives...

In [ ]:
# We have already loaded the subobjectives: penalty_on, penalty_off, penalty_cover!!
## And we have them stored in the list 'objectives'
non_dom_model = model.copy()

solutions_limit = 5
solutions = []
solution_images = []

while len(solutions) < solutions_limit:
    non_dom_model.solve() # find solution by optimizing again
    non_dom_model += cp.any([obj > obj.value() for obj in objectives])  # dominance constraint
    solutions.append(nurse_view.value())
    solution_images.append([obj.value() for obj in objectives])

for s in solutions:
    display(visualize(s, factory))



In [ ]:
# Create a DataFrame from the solution_images list
solutions_df = pd.DataFrame(
    solution_images,
    columns=['Penalty On', 'Penalty Off', 'Penalty Cover']
)

display(solutions_df)

## Deductive explanations

In the remainder of this notebook, we will explore different ways of explaining unsatisfiabily of the instance.

In [ ]:
model, nurse_view = factory.get_decision_model()
model.solve()

In [ ]:
from cpmpy.tools.explain import mus

t0 = time.time()
conflict = mus(model.constraints) # try different solvers here!
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

In [ ]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

Now, let's influence the MUS we would like to find.

We can chose from QuickXplain [1] or Optimal MUS (OUS) [2]

**QuickXplain** takes as input a total ordering of constraints, and returns a lexicographically minimal MUS.
The algorithm is build up as a divide-and-conquer approach, and therefore has a good average complexity.

**OUS** takes as input a weight for each constraint, and finds a **optimal** MUS. While this optimality guarantee is sometimes required, it comes at a penalty of longer computation times, as you will notice here!

In [ ]:
# QuickXplain first
from cpmpy.tools.explain import quickxplain


def get_weight(cons):
    if "William" in str(cons): # Find a different MUS than the previous
        return 2 
    return 1

ordered = sorted(model.constraints, key=get_weight)
conflict = quickxplain(ordered)
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

In [ ]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

In [ ]:
# Now find truely OPTIMAL MUSes
## Careful, this takes a while if you are not using Exact!
from cpmpy.tools.explain import optimal_mus

def get_weight(cons):
    if "william" in str(cons).lower():
        return 5
    else:
        return 1

solver = "exact" if "exact" in cp.SolverLookup.solvernames() else "ortools"
print("Using solver", solver)

conflict = optimal_mus(model.constraints, 
                       weights=[get_weight(c) for c in model.constraints],
                       solver=solver,
                       hs_solver="gurobi")
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

In [ ]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

## Part 2, fixing UNSAT models

Now that we know _why_ a model is UNSAT, we need to fix it.

In the presentation, several techniques are shown for doing so.

Below, you can find some skeleton code to play around with feasibiliy restoration techniques

In [ ]:
model, nurse_view = factory.get_decision_model()
model.solve()

In [ ]:
from cpmpy.tools.explain import mss_opt, mcs_opt

def get_weight(cons):
    if "cover" in str(cons):
        return 10
    return 1

# find Max-CSP solution
optimal_subset = mss_opt(model.constraints, hard=[],weights=[get_weight(c) for c in model.constraints])
mcs = set(model.constraints) - set(optimal_subset)
print("Found solution after dropping these constraints:")
for i,c in enumerate(mcs):
    print(f"{i}.", c)


In [ ]:
assert cp.Model(optimal_subset).solve() is True
visualize(nurse_view.value(), factory)
visualize_constraints(mcs, nurse_view, factory, do_clear=False)

### Slack-based relaxation

Apart from dropping constraints, they can also be _relaxed_ when numeric

In [ ]:
model, nurse_view, slack_under, slack_over = factory.get_slack_model()  # CMPpy Model

# TODO..

In [ ]:
import math

480 * 11